In [1]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from elasticsearch import Elasticsearch, exceptions

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1723637767679_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Elasticsearch configuration
es_host = "10.0.3.216"
es_port = 9200  # Ensure this is an integer
es_scheme = "http"
raw_es_index = "raw_stream_data"

raw_mappings = {
        "properties": {
            "timestamp": {
                "type": "date",
                "format": "strict_date_time"
            },
            "VIN": {
                "type": "keyword"
            },
            "ParameterName": {
                "type": "keyword"  # Flattened out from InternalParameter
            },
            "ParameterValue": {
                "type": "integer"  # Flattened out from InternalParameter
            },
            "ParameterUnit": {
                "type": "keyword"  # Flattened out from InternalParameter
            }
        }
    }


# Create Elasticsearch client
es = Elasticsearch([{'host': es_host, 'port': es_port, 'scheme': es_scheme}])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Kafka parameters
kafka_bootstrap_servers = "b-2.kafkacluster99.c5pq03.c3.kafka.eu-central-1.amazonaws.com:9092,b-1.kafkacluster99.c5pq03.c3.kafka.eu-central-1.amazonaws.com:9092"
kafka_topic = "topic1"

# Elasticsearch configuration
es_host = "10.0.3.216"
es_port = 9200  # Ensure this is an integer
es_scheme = "http"
raw_es_index = "raw_stream_data"

raw_mappings = {
        "properties": {
            "timestamp": {
                "type": "keyword",
            },
            "VIN": {
                "type": "keyword"
            },
            "ParameterName": {
                "type": "keyword"  # Flattened out from InternalParameter
            },
            "ParameterValue": {
                "type": "integer"  # Flattened out from InternalParameter
            },
            "ParameterUnit": {
                "type": "keyword"  # Flattened out from InternalParameter
            }
        }
    }


# Create Elasticsearch client
es = Elasticsearch([{'host': es_host, 'port': es_port, 'scheme': es_scheme}])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Define the schema
schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("VIN", StringType(), True),
    StructField("InternalParameter", StructType([
        StructField("ParameterName", StringType(), True),
        StructField("ParameterValue", IntegerType(), True),
        StructField("ParameterUnit", StringType(), True)
    ]), True)
])

# Kafka parameters
kafka_bootstrap_servers = "b-2.kafkacluster99.c5pq03.c3.kafka.eu-central-1.amazonaws.com:9092,b-1.kafkacluster99.c5pq03.c3.kafka.eu-central-1.amazonaws.com:9092"
kafka_topic = "topic1"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def create_index_if_not_exists(es_client, index_name, mappings):
    try:
        if not es_client.indices.exists(index=index_name):
            print(f"Index '{index_name}' does not exist. Creating index...")
            es_client.indices.create(
                index=index_name,
                body={
                    "mappings": mappings  # Use the provided mappings parameter
                }
            )
            print(f"Index '{index_name}' created successfully.")
        else:
            print(f"Index '{index_name}' already exists.")
    except exceptions.RequestError as e:
        print(f"RequestError: {e.info}")
    except exceptions.ConnectionError as e:
        print(f"ConnectionError: {e}")
    except Exception as e:
        print(f"Error creating index: {e}")
        
# Create the index in Elasticsearch
create_index_if_not_exists(es, raw_es_index, raw_mappings)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index 'raw_stream_data' already exists.

In [ ]:
# Create SparkSession
spark = SparkSession.builder \
    .appName("KafkaToElasticsearch") \
    .config("spark.sql.streaming.checkpointLocation", "s3://aws-emr-studio-381492251123-eu-central-1/stream_checkpoint/checkpoint/") \
    .getOrCreate()

# Create a DataFrame representing the stream of input lines from Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

# Convert the value column from Kafka to a string
df = df.selectExpr("CAST(value AS STRING)")

# Parse the JSON data using the schema
df = df.select(from_json(col("value"), schema).alias("data"))

# Keep the InternalParameter as a nested object
df = df.select(
    col("data.timestamp"),
    col("data.VIN"),
    col("data.InternalParameter.ParameterName"),
    col("data.InternalParameter.ParameterValue"),
    col("data.InternalParameter.ParameterUnit")  
)

# Elasticsearch configuration for Spark
es_write_conf = {
    "es.nodes": es_host,
    "es.port": str(es_port),  # Convert port to string for Spark configuration
    "es.index.auto.create": "true"  # Ensure that Spark does not attempt to auto-create the index
}

# Write data to Elasticsearch
query = df.writeStream \
    .format("org.elasticsearch.spark.sql") \
    .options(**es_write_conf) \
    .option("es.resource", raw_es_index) \
    .outputMode("append") \
    .start()

# Await termination of the stream
query.awaitTermination()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…